## 動画による画像収集と教師なし学習によるラベリング

動画を用いて画像データを効率的に集める方法と効率的にラベリングする方法を学びます。  


以下、流れになります。  
①画像データを効率的に集めるため、動画からフレーム毎に顔画像をトリミングし保存（openCVという画像認識ライブラリを使用します。）  
②ラベリングを効率的に行うため、教師なし学習により分類しラベリング  

使用するライブラリをインストールします。（他に必要なライブラリがあれば個別にインストールしてください）

In [ ]:
#open cvのインストール
! pip install opencv-python

In [ ]:
#scipyのインストール
! pip install scipy

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.misc
import glob

まずopenCVで使用するカスケードファイルとそのファイルの場所を指定します。今回はhaarcascade_frontalface_alt2.xmlというカスケードファイルを使います。このファイルを使用することで顔と顔でないものを分類することが可能となります。  
以下のコマンドを実行してください。ご自身のパソコン内のカスケードファイルの場所が表示されます。

In [ ]:
! mdfind haarcascade_frontalface_alt2.xml

上で表示された場所（/Users/yuza/anaconda3/lib/python3.6/site-packages/cv2/data/haarcascade_frontalface_alt2.xmlに似ている行）をコピーし、下の部分を置き換えてください。

In [ ]:
#cascadeファイルのパス指定
cascade_path = "/Users/yuza/anaconda3/lib/python3.6/site-packages/cv2/data/haarcascade_frontalface_alt2.xml"

## 動画から顔画像のトリミング

今回は簡単な例としてデータミックス のスクール紹介動画の１部分を使用します。  
openCVを使用すると顔を自動的に認識でき顔部分の座標が取得できますので、それを元にトリミングし保存します。また元画像はRGBですが、色による影響を少なくするために白黒に変換します。 
トリムした画像はtrim_imageという名前のフォルダに保存します。

In [ ]:

#画像から顔認識
def detect_face(image,cascade):
    facerect = cascade.detectMultiScale(
        image,
        scaleFactor=1.5,
        minNeighbors=3,
        minSize=(40, 40))
    return facerect
  
#顔画像のみトリムし保存
def trim_face_and_save(facerect,image,file_path,file_name,count):
    if 0 != len(facerect):
        for i,r in enumerate(facerect):
            x = r[0]
            y=r[1]
            width = r[2]
            height = r[3]
            cut_image = image[y:y+height,x:x+width]
            savepath = "{}/{}_{}_{}.png".format(file_path, file_name,count,i)
            cv2.imwrite(savepath,cut_image)


def addrectangle_to_pic(image,facerect):
    color = (255, 255, 255)
    for rect in facerect:
        cv2.rectangle(image, tuple(rect[0:2]),tuple(rect[0:2]+rect[2:4]), color, thickness=2)
        
#動画から顔のみトリムし画像保存
def trim_from_video(cascade,file_path):
    count = 0
    cap = cv2.VideoCapture(ORG_FILE_NAME)
    while True:
        ret, frame = cap.read()
        if ret == True:   
            facerect = detect_face(frame,cascade)
            if 0 != len(facerect):
                count += 1
                print("face deteted! total_num = {0}".format(count))
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                trim_face_and_save(facerect,gray,file_path,"trim",count)
        else:
            break

            
#対象の動画ファイルパス指定
ORG_FILE_NAME = "./movie/sample_datamix.mp4"

#対象の動画ファイルパス指定
cascade = cv2.CascadeClassifier(cascade_path)

#保存フォルダ名指定
folder_name = "trim_image"
file_path = "./{}".format(folder_name)

#フォルダ作成
try:
    os.mkdir("{}".format(folder_name))
except:
    pass

trim_from_video(cascade,file_path)
print("trim finished!")

実行を終えるとtrim_imageというフォルダの下に顔のみトリムされた画像が配置されています。

## 画像の配列化

トリムした画像を配列として読み込みます。データのサイズを揃えるため、一定の解像度に落とします。

In [ ]:
#白黒反転
def bitwise(data):
    data = cv2.bitwise_not(data)
    return data

count = 0
list_img_vector = []

#トリムした顔画像pngファイルを読み込む
list_images = glob.glob('{}/*.png'.format(file_path))
base_list_images = [os.path.basename(r) for r in glob.glob('{}/*.png'.format(file_path))]


#読み込んだ画像を配列化する
for f in list_images:
    img = cv2.imread('{}'.format(f),0)
    img = bitwise(img)
    vector = np.asarray(img/255)
    #画像を100×100に解像度を落とし、揃えます。
    vector = scipy.misc.imresize(vector, (100, 100))
    list_img_vector.append(vector)
    array_img_vector = np.array(list_img_vector)

print(array_img_vector.shape)

#     plt.imshow(vector.reshape(28,28),cmap="gray_r")
#     plt.show()

## ラベリングのための教師なし学習による分類

教師あり学習を行うためには学習データにラベリングが必要ですが、その作業を効率化するため、今回は教師なし学習を用います。（他にラベリングを効率化する手段としてアクティブラーニングや半教師あり学習があります。）
まずTsneによる次元削減を行い、その後Kmeansによるクラスタリングを行います。

In [ ]:
#Tsneによる次元削減
from sklearn.manifold import TSNE

#画像をフラットな配列に変換します
reshaped_array = np.reshape(array_img_vector,[-1,100*100])
reduced = TSNE(n_components=2, random_state=0).fit_transform(reshaped_array)

plt.scatter(reduced[:, 0], reduced[:, 1])
plt.show()


大きく分けて２つに分類できそうです。

In [ ]:
#念のためエルボー法でSSEを確認

from sklearn.cluster import KMeans
SSE = []
for i in range(1,20):
    km = KMeans(n_clusters=i,random_state=0)
    pred = km.fit_predict(reduced)
    SSE.append(km.inertia_)
    
plt.plot(range(1,20),SSE,marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.show()

クラスタ数を２としてKmeansにより分類してみましょう。

In [ ]:
#クラスタ数を決定しクラスタリングを行う
km = KMeans(n_clusters=2,random_state=0)
pred = km.fit_predict(reduced)

In [ ]:
#どのように分類されたか可視化
from matplotlib.colors import ListedColormap

target_names = set(pred)
colors = str('rgbcmykw')
target_ids = range(len(target_names))
plt.figure()
for i, c, label in zip(target_ids, colors, target_names):
    plt.scatter(reduced[pred == i, 0], reduced[pred == i, 1],
                    c=c, label=label)
plt.legend()
plt.show()


得られたラベルの情報を元に画像のファイルをフォルダに振り分けます。
今回は今までの解像度を落とした画像を保存するのではなく、確認を容易にするため元の解像度の画像を保存します。

In [ ]:
#分類結果を元に画像をそれぞれフォルダに振り分ける
import shutil

array_file_pred = np.vstack([np.array(pred), np.array(base_list_images)]).T
dic_file_pred = {}

for i in set(pred):
    try:
        os.mkdir("./trim_image/cluster_{}".format(str(i)))
    except:
        pass
    
    for f in array_file_pred[:,1][array_file_pred[:,0]==str(i)]:
        shutil.move("./trim_image/{}".format(f), "./trim_image/cluster_{}".format(str(i)))
        
    

振り分けられたフォルダの中身を確認してみると、（完全に正確な訳ではないですが）ほぼ正確に分類されていることがわかります。  
もし誤分類されているデータを見つけたら消去してしまいましょう。

## ラベリングのための教師なし学習による分類
次に上で行なった振り分けを元にデータセットを作成します。

In [ ]:
img_folder = "{}/trim_image".format(os.getcwd())
folders = os.listdir(img_folder)

X_train =[]
y_train = []

for folder in folders:
    if "cluster_" in folder:
        list_images = glob.glob('{}/{}/*.png'.format(img_folder,folder))
        for f in list_images:
            img = cv2.imread('{}'.format(f),0)
            img = bitwise(img)
            vector = np.asarray(img)
            #以下で解像度を落としています
            vector = scipy.misc.imresize(vector, (100, 100))
            X_train.append(vector/255)
            
            cluster_index = folder.replace("cluster_","") 
            y_train.append(cluster_index) 

#画像を100×100に解像度を落とし、揃えます。
X_train = np.concatenate(X_train).reshape(-1,100,100)
y_train = np.array(y_train)

print(X_train.shape)
print(y_train.shape)

これで教師あり学習に用いるデータセットの作成ができました。